<a href="https://colab.research.google.com/github/alfekka/datasets_ABBYY/blob/main/pos_tagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymorphy2
!pip install russian_tagsets
!pip install pos

ERROR: Could not find a version that satisfies the requirement pos (from versions: none)
ERROR: No matching distribution found for pos


In [ ]:
import nltk
nltk.download('punkt')
import pandas as pd
from nltk.corpus import wordnet
from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from nltk.corpus import PlaintextCorpusReader
from nltk.stem.snowball import SnowballStemmer
from nltk.probability  import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk import bigrams
from nltk import pos_tag
from collections import OrderedDict
from sklearn.metrics import classification_report, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
from multiprocessing import Pool
import numpy as np
from scipy.sparse import csr_matrix
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_ru.zip.


True

In [ ]:
!pip install emoji
import emoji

     |████████████████████████████████| 133kB 3.8MB/s 


In [ ]:
df = pd.read_csv('/content/drive/My Drive/Corpus_project/basic_cleaning_tweets.csv', delimiter=',', quotechar='"')
df

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,id,date,tweet
0,0,1323414585366450176,2020-11-02 23:59:59+00:00,подружка скинула песни алены швец теперь лежу ...
1,1,1323414583663579141,2020-11-02 23:59:59+00:00,Лол ивент рашки окончен. Из сотки сундуков мне...
2,2,1323414582162034695,2020-11-02 23:59:58+00:00,Юи солнышко
3,3,1323414578055794691,2020-11-02 23:59:57+00:00,Дайте поспааать
4,4,1323414572162830339,2020-11-02 23:59:56+00:00,хочу ему написать..
...,...,...,...,...
702799,702799,1211021163725512704,2019-12-28 20:28:39+00:00,РоссияКанада 6:0 ! Молодцы ! Завтра матч с аме...
702800,702800,1211021160290365441,2019-12-28 20:28:38+00:00,Так это...а можно надергать из толпы противник...
702801,702801,1211021151385923584,2019-12-28 20:28:36+00:00,"молодцы наши, но главные матчи еще впереди)"
702802,702802,1211021149754277889,2019-12-28 20:28:36+00:00,Что то я не нашла дюти фри...


In [ ]:
pos_stats = nltk.FreqDist()
stemmer = nltk.stem.snowball.SnowballStemmer('russian')

In [ ]:
def give_emoji_free_text(text):
    # allchars = text
    # emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    # clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    clean_text = re.sub('&gt;', '', text)
    return clean_text

In [ ]:
def lower_pos_tag(words):
    lower_words = []
    for i in words:
        lower_words.append(i.lower())
    pos_words = pos_tag(lower_words, lang='rus')
    return pos_words

In [ ]:
def sentences_tokenization(data):
  proverbs = []
  for s in data:
    # print(type(s))
    p = str(s)
    p = re.sub('&gt;', '', p)
    p = re.sub('  ', ' ', p)
    p = re.sub('``', '', p)
    proverbs.append(p)
  # proverbs = data
  with open('/content/drive/My Drive/Corpus_project/pos_tagging.tags', 'w', encoding='utf-8') as dst:
    for d in proverbs:
      d = re.sub('&gt;', '', d)
      d = re.sub('  ', ' ', d)
      d = re.sub('``', '', d)
      offset = 0
      lower_proverb : str = d.lower()
      sentences = nltk.sent_tokenize(d)
      print(d)
      for sentence in sentences:
        print(sentence, file=dst)
        words = nltk.word_tokenize(sentence)

        pos_words = lower_pos_tag(words)
        pos_stats.update(tag for word, tag in pos_words)
      


        for word, tag in pos_words:
          word_offset = lower_proverb.find(word.lower(), offset)
          lemmas = stemmer.stem(word)
          print(word, word_offset, lemmas, tag, sep = '\t', file=dst)
          offset = word_offset + len(word)
          print(file=dst)
      print(file=dst)

  dst.close()
  return(pos_stats)




In [ ]:
type(data[0])

str

In [ ]:
sentences_tokenization(data)

Выходные данные были обрезаны до нескольких последних строк (5000).
Почти заказала открытку с гороми чтобы повесить на работе, но пожалела хрупкий разум коллег Гороми слишком хороша и всех отвлечет
кто выпить хочет? открыт для всех предложений (на потусить) в конце декабря
Не хочу чтоб корешей паковали в цирк
очевидно что у меня сейчас комплит брэйкдаун.. я плачу по куче часов в день и у меня ноль сил и наверное это сочетание пмс работы отсутствия каникул стресса из-за заказов и из-за нового года я просто хочу отдохнуть...
Полный балаган
Я сейчас просто панически бъюсь головой в стену к соседям, ибо: а) эти мудаки шумят, Новый Год, блять, празднуют; б) стыдно, за то, что наговорил в диалоге, с одним хорошим человеком в) прости, жизнь, пиздец, как стыдно
лрт ЭТО??? ТАК???? СОФТ???? Я ЛЮБЛЮ ВАС
Ля, ещё седьмую серию гляну и спать Клянусь
В вк к голосовым нужно добавить функцию ПРОСЛУШАТЬ ПОЗЖЕ чтоб они откладывались в отдельной странице и ты находясь например в караоке, когда не можешь п

FreqDist({'A': 12032,
          'A-PRO': 17817,
          'A-PRO=dat': 6,
          'A-PRO=f': 142423,
          'A-PRO=m': 192991,
          'A-PRO=n': 62062,
          'A-PRO=pl': 193274,
          'A-PRO=sg': 14199,
          'A=brev': 1835,
          'A=comp': 22838,
          'A=comp2': 1931,
          'A=comp=anom': 5,
          'A=f': 180976,
          'A=gen': 27,
          'A=m': 284136,
          'A=n': 115936,
          'A=pl': 172532,
          'A=plen': 73,
          'A=sg': 12817,
          'ADV': 639487,
          'ADV-PRO': 384302,
          'ADV-PRO=abbr': 451,
          'ADV-PRO=anom': 13,
          'ADV-PRO=comp': 4926,
          'ADV-PRO=comp2': 143,
          'ADV-PRO=distort': 8982,
          'ADV=abbr': 251,
          'ADV=comp': 17526,
          'ADV=comp2': 2136,
          'ADV=distort': 20,
          'ANUM': 2,
          'ANUM=ciph': 594,
          'ANUM=f': 11455,
          'ANUM=gen': 14,
          'ANUM=m': 25610,
          'ANUM=n': 3485,
          'ANUM=n

In [ ]:
with open('/content/drive/My Drive/Corpus_project/pos_statistics.txt' , 'w', encoding='utf-8') as dst:
      print('Статистика по частям речи в корпусе:')
      for i, (pos, freq) in enumerate(pos_stats.most_common()):
          print(f'{i}. {pos} -> {freq} ({freq / pos_stats.N():.4f})')
          print(pos, freq, sep='\t', file=dst)

Статистика по частям речи в корпусе:
0. S -> 3665184 (0.2227)
1. V -> 2539788 (0.1543)
2. NONLEX -> 2527006 (0.1535)
3. S-PRO -> 1455938 (0.0885)
4. PR -> 1344246 (0.0817)
5. CONJ -> 1112312 (0.0676)
6. PART -> 742275 (0.0451)
7. ADV -> 639487 (0.0389)
8. ADV-PRO -> 384302 (0.0233)
9. A=m -> 284136 (0.0173)
10. NUM=ciph -> 214412 (0.0130)
11. A-PRO=pl -> 193274 (0.0117)
12. A-PRO=m -> 192991 (0.0117)
13. A=f -> 180976 (0.0110)
14. A=pl -> 172532 (0.0105)
15. A-PRO=f -> 142423 (0.0087)
16. PRAEDIC -> 134960 (0.0082)
17. A=n -> 115936 (0.0070)
18. A-PRO=n -> 62062 (0.0038)
19. INTJ -> 32264 (0.0020)
20. NUM=acc -> 27755 (0.0017)
21. ANUM=m -> 25610 (0.0016)
22. S-PRO=acc -> 23006 (0.0014)
23. A=comp -> 22838 (0.0014)
24. PARENTH -> 20796 (0.0013)
25. NUM=nom -> 19537 (0.0012)
26. A-PRO -> 17817 (0.0011)
27. ADV=comp -> 17526 (0.0011)
28. S-PRO=dat -> 15037 (0.0009)
29. A-PRO=sg -> 14199 (0.0009)
30. NUM=m -> 12905 (0.0008)
31. A=sg -> 12817 (0.0008)
32. A -> 12032 (0.0007)
33. ANUM=f -> 